# Profitable App for the Google and Apple Store
Our aim in this analysis is to find the most used apps in both the Google and Apple app stores. The most used apps tend to be the most profitable, and investing in the creation of a certain type of app could yield a higher return on investment.

In [2]:
import csv

def get_data(file_name):
    with open(file_name) as file:
        reader = csv.reader(file)
        data = list(reader)
        return data

apple_data = get_data("AppleStore.csv")
google_data = get_data("googleplaystore.csv")

In [1]:
def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]    
    for row in dataset_slice:
        print(row)
        print('\n') # adds a new (empty) line after each row

    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))

User reported issue with entry 10472 (assuming the header row is not included) https://www.kaggle.com/lava18/google-play-store-apps/discussion/66015 .
We'll delete this corrupted entry.

In [3]:
del google_data[10473]

Looking further, we can see that the google dataset has duplicate entries. There are 1,181 total duplicates. To show an example, "8 Ball Pool" has 6 duplicates of itself.

In [4]:
def count_duplicates(data_set):
    a_dict = {}
    dup_entries = []

    for data in data_set:
        name = data[0]
        
        if name in a_dict:
            dup_entries.append(data)
        else:
            a_dict[name] = ""
            
    dup_entries.sort()
    for dup in dup_entries[6:12]:
        print(dup)
        print("\n")
    print("Number of duplicates: " + str(len(dup_entries)))
    
count_duplicates(google_data)

['8 Ball Pool', 'GAME', '4.5', '14198602', '52M', '100,000,000+', 'Free', '0', 'Everyone', 'Sports', 'July 31, 2018', '4.0.0', '4.0.3 and up']


['8 Ball Pool', 'GAME', '4.5', '14200344', '52M', '100,000,000+', 'Free', '0', 'Everyone', 'Sports', 'July 31, 2018', '4.0.0', '4.0.3 and up']


['8 Ball Pool', 'GAME', '4.5', '14200550', '52M', '100,000,000+', 'Free', '0', 'Everyone', 'Sports', 'July 31, 2018', '4.0.0', '4.0.3 and up']


['8 Ball Pool', 'GAME', '4.5', '14201604', '52M', '100,000,000+', 'Free', '0', 'Everyone', 'Sports', 'July 31, 2018', '4.0.0', '4.0.3 and up']


['8 Ball Pool', 'GAME', '4.5', '14201891', '52M', '100,000,000+', 'Free', '0', 'Everyone', 'Sports', 'July 31, 2018', '4.0.0', '4.0.3 and up']


['8 Ball Pool', 'SPORTS', '4.5', '14184910', '52M', '100,000,000+', 'Free', '0', 'Everyone', 'Sports', 'July 31, 2018', '4.0.0', '4.0.3 and up']


Number of duplicates: 1181


Looking into these duplicates, we can see that some things differ. The most obvious difference being the number of ratings amongst all of the entries for "8 Ball Pool". The number of ratings can indicate which entry is the most recent. We should only capture the duplicate with the highest number of ratings and delete the rest.

In [5]:
def remove_google_duplicates():
    reviews_max = {}
    # create dictionary
    # {key=unique_app_name, value=highest_number_of_ratings})
    for data in google_data[1:]:
        name = data[0]
        n_reviews = float(data[3])
        
        if name in reviews_max and reviews_max[name] < n_reviews:
            reviews_max[name] = n_reviews
        elif name not in reviews_max:
            reviews_max[name] = n_reviews
    # android_clean contains non-duplicated data 
    # already_added contains duplicated data
    android_clean = []
    already_added = []
    
    for data in google_data[1:]:
        name = data[0]
        n_reviews = float(data[3])
        
        if n_reviews == reviews_max[name] and name not in already_added:
            android_clean.append(data)
            already_added.append(name)
    # returns non-duplicated data    
    return android_clean
        
google_data = remove_google_duplicates()

Filter data to obtain English named, free apps.

In [6]:
# check if more than 3 characters fall outside the 
# ASCII range(0 = 127)
# (check if string is non-English)
def char_check(string):
    not_eng = 0
    
    for char in string:
        if ord(char) > 127:
            not_eng += 1
            if not_eng == 4:
                return False

    return True

In [7]:
def filter_non_eng(data_set):
    is_eng = []
    
    if data_set[0][0].isnumeric():
    
        for data in data_set:
            
            if char_check(data[1]):
                is_eng.append(data)
    else:
        
        for data in data_set:
        
            if char_check(data[0]):
                is_eng.append(data)
            
    return is_eng

google_data = filter_non_eng(google_data)
apple_data = filter_non_eng(apple_data[1:])

In [8]:
def filter_free(data_set):
    is_free = []
    
    for data in data_set:
        
        if data[7] == "0" or data[4] == "0.0":
            is_free.append(data)
    
    return is_free

google_data = filter_free(google_data)
apple_data = filter_free(apple_data)

If we were to develop an app using the information obtained from the dataset, the app would have to be alluring to users from the google play store AND the apple store. The validation strategy for an app idea would be comprised of three steps:

1. Build a minimal Android version of the app, and add it to Google Play.

2. If the app has a good response from users, we develop it further.

3. If the app is profitable after six months, we build an iOS version of the app and add it to the Apple Store.

In [9]:
def freq_table(dataset, index):
    table = {}
    
    for data in dataset:
        item_at_column = data[index]
        
        if item_at_column in table:
            table[item_at_column] += 1
        else:
            table[item_at_column] = 1
            
    for item in table:
        table[item] = table[item] / len(dataset) * 100
        
    return table

In [10]:
def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])

In [11]:
display_table(apple_data, -5)

Games : 58.16263190564867
Entertainment : 7.883302296710118
Photo & Video : 4.9658597144630665
Education : 3.662321539416512
Social Networking : 3.2898820608317814
Shopping : 2.60707635009311
Utilities : 2.5139664804469275
Sports : 2.1415270018621975
Music : 2.0484171322160147
Health & Fitness : 2.0173805090006205
Productivity : 1.7380509000620732
Lifestyle : 1.5828677839851024
News : 1.3345747982619491
Travel : 1.2414649286157666
Finance : 1.1173184357541899
Weather : 0.8690254500310366
Food & Drink : 0.8069522036002483
Reference : 0.5586592178770949
Business : 0.5276225946617008
Book : 0.4345127250155183
Navigation : 0.186219739292365
Medical : 0.186219739292365
Catalogs : 0.12414649286157665


In [12]:
display_table(google_data, 9)

Tools : 8.449909747292418
Entertainment : 6.069494584837545
Education : 5.347472924187725
Business : 4.591606498194946
Productivity : 3.892148014440433
Lifestyle : 3.892148014440433
Finance : 3.7003610108303246
Medical : 3.531137184115524
Sports : 3.463447653429603
Personalization : 3.3167870036101084
Communication : 3.2378158844765346
Action : 3.1024368231046933
Health & Fitness : 3.0798736462093865
Photography : 2.944494584837545
News & Magazines : 2.7978339350180503
Social : 2.6624548736462095
Travel & Local : 2.3240072202166067
Shopping : 2.2450361010830324
Books & Reference : 2.1435018050541514
Simulation : 2.0419675090252705
Dating : 1.861462093862816
Arcade : 1.8501805054151623
Video Players & Editors : 1.7712093862815883
Casual : 1.7599277978339352
Maps & Navigation : 1.3989169675090252
Food & Drink : 1.2409747292418771
Puzzle : 1.128158844765343
Racing : 0.9927797833935018
Role Playing : 0.9363718411552346
Libraries & Demo : 0.9363718411552346
Auto & Vehicles : 0.9250902527075

In [13]:
display_table(google_data, 1)

FAMILY : 18.907942238267147
GAME : 9.724729241877256
TOOLS : 8.461191335740072
BUSINESS : 4.591606498194946
LIFESTYLE : 3.9034296028880866
PRODUCTIVITY : 3.892148014440433
FINANCE : 3.7003610108303246
MEDICAL : 3.531137184115524
SPORTS : 3.395758122743682
PERSONALIZATION : 3.3167870036101084
COMMUNICATION : 3.2378158844765346
HEALTH_AND_FITNESS : 3.0798736462093865
PHOTOGRAPHY : 2.944494584837545
NEWS_AND_MAGAZINES : 2.7978339350180503
SOCIAL : 2.6624548736462095
TRAVEL_AND_LOCAL : 2.33528880866426
SHOPPING : 2.2450361010830324
BOOKS_AND_REFERENCE : 2.1435018050541514
DATING : 1.861462093862816
VIDEO_PLAYERS : 1.7937725631768955
MAPS_AND_NAVIGATION : 1.3989169675090252
FOOD_AND_DRINK : 1.2409747292418771
EDUCATION : 1.1620036101083033
ENTERTAINMENT : 0.9589350180505415
LIBRARIES_AND_DEMO : 0.9363718411552346
AUTO_AND_VEHICLES : 0.9250902527075812
HOUSE_AND_HOME : 0.8235559566787004
WEATHER : 0.8009927797833934
EVENTS : 0.7107400722021661
PARENTING : 0.6543321299638989
ART_AND_DESIGN : 

Let us analyze the average number of user ratings for each app. Analyzing the number of installs would have been better as it's known that not everyone who installs an app also rates it. Number of installs cannot be found in the apple dataset however, and so going over the number of ratings is the next best thing.

In [22]:
def avg_ratings(table=freq_table(apple_data, -5), data_set=apple_data):
    
    for genre in table:
        total = 0
        len_genre = 0
        
        for data in data_set:
            genre_app = data[-5]
            
            if genre_app == genre:
                total += float(data[5])
                len_genre += 1
                
        avg_num_ratings = total / len_genre
        
        print("Genre: " + genre + " || " +
              "Avg. Number of User Ratings: " + str(avg_num_ratings))
        print()

avg_ratings()

Genre: Catalogs || Avg. Number of User Ratings: 4004.0

Genre: Social Networking || Avg. Number of User Ratings: 71548.34905660378

Genre: News || Avg. Number of User Ratings: 21248.023255813954

Genre: Shopping || Avg. Number of User Ratings: 26919.690476190477

Genre: Health & Fitness || Avg. Number of User Ratings: 23298.015384615384

Genre: Navigation || Avg. Number of User Ratings: 86090.33333333333

Genre: Music || Avg. Number of User Ratings: 57326.530303030304

Genre: Games || Avg. Number of User Ratings: 22788.6696905016

Genre: Utilities || Avg. Number of User Ratings: 18684.456790123455

Genre: Entertainment || Avg. Number of User Ratings: 14029.830708661417

Genre: Book || Avg. Number of User Ratings: 39758.5

Genre: Sports || Avg. Number of User Ratings: 23008.898550724636

Genre: Reference || Avg. Number of User Ratings: 74942.11111111111

Genre: Productivity || Avg. Number of User Ratings: 21028.410714285714

Genre: Business || Avg. Number of User Ratings: 7491.117647058

Social Networking apps seem to be the most popular on the Apple Store.

Genre: Social Networking || Avg. Number of User Ratings: 
71,548.34905660378

Fortunately, the google dataset does contain information for the number of installs. Unfortunately, though, it does not contain exact amounts. The information resembles the likes of "100,000+" or "1,000,000+". We can still sdjust to this by changing "100,000+" to "100,000" and getting the average number of installs, but the information will be slightly innacurate.

In [26]:
def avg_installs(table=freq_table(google_data, 1), data_set=google_data):
    
    for category in table:
        total = 0
        len_category = 0
        
        for data in data_set:
            category_app = data[1]
            
            if category_app == category:
                installs = data[5].replace("+", "").replace(",", "")
                total += float(installs)
                len_category += 1
                
        avg_num_installs = total / len_category
        
        print("Category: " + category + " || " +
              "Number of Installs: " + str(avg_num_installs))
        print()

avg_installs()

Category: LIFESTYLE || Number of Installs: 1437816.2687861272

Category: DATING || Number of Installs: 854028.8303030303

Category: PARENTING || Number of Installs: 542603.6206896552

Category: HEALTH_AND_FITNESS || Number of Installs: 4188821.9853479853

Category: COMICS || Number of Installs: 817657.2727272727

Category: WEATHER || Number of Installs: 5074486.197183099

Category: HOUSE_AND_HOME || Number of Installs: 1331540.5616438356

Category: SHOPPING || Number of Installs: 7036877.311557789

Category: BOOKS_AND_REFERENCE || Number of Installs: 8767811.894736841

Category: BEAUTY || Number of Installs: 513151.88679245283

Category: EVENTS || Number of Installs: 253542.22222222222

Category: EDUCATION || Number of Installs: 1833495.145631068

Category: PHOTOGRAPHY || Number of Installs: 17840110.40229885

Category: FAMILY || Number of Installs: 3695641.8198090694

Category: VIDEO_PLAYERS || Number of Installs: 24727872.452830188

Category: FOOD_AND_DRINK || Number of Installs: 192

Communication and Social apps seem to be the most popular on the Google Store.

Category: SOCIAL || Number of Installs: 23,253,652.127118643
Category: COMMUNICATION || Number of Installs: 38,456,119.167247385

The most attractive app to build would have to be in the category or genre of social media. These types get the most amount of installs and user ratings and is popular on both the google and apple store.